In [18]:
import sys
sys.path.append('/data/code/finhack')
import pandas as pd
from factors.indicatorCompute import indicatorCompute

 

mypath='/data/code/finhack'
cache_path_25=mypath+"/cache/factors/mining_factor_df_25"

if os.path.isfile(cache_path_25):
    df_all_25=pd.read_pickle(cache_path_25)
else:
    factor_list=['buySmVol_0','WILLR_0']
    df_all_25=indicatorCompute.computeListByStock(ts_code='test',list_name='test',where='',factor_list=factor_list,c_list=[],pure=True,check=True,df_price=df_25,db='tushare')
    df_all_25=df_all_25.dropna()
    df_all_25['Y']=df_all_25.groupby('ts_code')['close'].apply(lambda x: x/x.shift(10))
    df_all_25.to_pickle(cache_path_25)      
df_all_25=df_all_25.dropna()
df_all_25=df_all_25.reset_index(drop=True)
print(df_all_25)

      trade_date    ts_code       open       high        low      close  \
0       20161101  001979.SZ   17.14312   17.34672   17.08204   17.31618   
1       20161102  001979.SZ   17.29582   17.29582   16.72574   16.78682   
2       20161103  001979.SZ   16.79700   17.08204   16.74610   16.85808   
3       20161104  001979.SZ   16.89880   17.08204   16.77664   16.83772   
4       20161107  001979.SZ   16.89880   16.92934   16.72574   16.75628   
...          ...        ...        ...        ...        ...        ...   
25713   20200716  688111.SH  414.96455  418.38797  360.47011  375.24487   
25714   20200717  688111.SH  372.57220  376.15578  344.23389  348.99865   
25715   20200720  688111.SH  355.36501  357.35700  322.32200  353.55320   
25716   20200721  688111.SH  354.35400  376.25588  354.35400  369.00864   
25717   20200722  688111.SH  370.37000  398.39800  365.55519  384.37399   

       pre_close    change  returns     volume       amount adj_factor  \
0       17.17366   0.1425

In [20]:
from sklearn.datasets import load_boston
from gplearn.genetic import SymbolicTransformer
import gplearn as gp

label = df_all_25['Y']
train = df_all_25.drop(columns=['ts_code','trade_date','change','pre_close','adj_factor','Y'])
init_function = ['add', 'sub', 'mul', 'div', 'sqrt', 'abs', 'sin', 'cos', 'tan']
gp1 = SymbolicTransformer(  
                            generations=5, #整数，可选(默认值=20)要进化的代数
                            population_size=1000,# 整数，可选(默认值=1000)，每一代群体中的公式数量
                            hall_of_fame=100, 
                            n_components=100,#最终生成的因子数量
                            function_set=init_function ,
                            parsimony_coefficient=0.0005,
                            max_samples=0.9, 
                            verbose=1,
                            const_range = (10.0,20.0),
                            feature_names=list('$'+n for n in train.columns),
                            random_state=int(time.time()),  # 随机数种子
                            n_jobs=-1
                    )

gp1.fit(train,label)
print(gp1)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     9.12        0.0147256        9         0.196963        0.0435231      1.43s
   1     6.98        0.0557384        7         0.195568         0.072883      2.82s
   2     6.94        0.0962457       12         0.200578        0.0138036      2.44s
   3     7.34         0.106108        8         0.202502       0.00258283      1.22s
   4     7.66         0.117763       14         0.204924        0.0284698      0.00s
[sqrt(div(abs($returns), add(mul(abs($high), sub($amount, sqrt($returns))), sqrt($volume)))),
 div(abs($returns), add(mul($amount, $high), $amount)),
 cos(sqrt(sqrt(sin(cos(sqrt(sqrt(div(abs($returns), div(add($amount, $open), abs(10.729)))))))))),
 div(div(sqrt($buySmVol_0), $vwap), add(abs($amount), $vwap)),
 div(10.682, sin(c

In [31]:
import factors.alphaEngine as alphaFunc

df_tmp=df_all_25[['ts_code','trade_date']]
df_tmp=df_tmp.reset_index(drop=True)

def trans_xy(xy,key='x'):
    status=True
    if(len(xy)<100):
        status=False
        return status,xy
    if not 'numpy' in str(type(xy)):
        xy=np.zeros(len(xy))
        status=False
        return status,xy
    if 'numpy.memmap' in str(type(xy)):
        xy=np.array(xy)    
    if(type(xy)==type(np.ndarray([]))):
        df_xy=df_tmp.copy()
        df_xy[key]=xy
        xy=df_xy
        xy=xy.set_index(['ts_code','trade_date'])
        xy=xy[key]   
    return status,xy

def _log(x):
    if x.max()==x.min() and x.mean()==0:
        return np.zeros(len(x))
    status,x=trans_xy(x,'x')
    if not status:
        return x
    df=alphaFunc.log(x)
    return np.nan_to_num(df.values)   

def _ts_sum(x):
    status,x=trans_xy(x,'x')
    if not status:
        return x
    df=alphaFunc.ts_sum(x)
    return np.nan_to_num(df.values)
    
def _delta(x):
    status,x=trans_xy(x,'x')
    if not status:
        return x
    df=alphaFunc.delta(x)
    return np.nan_to_num(df.values) 

def _correlation(x,y):
    status,x=trans_xy(x,'x')
    if not status:
        return x
    status,y=trans_xy(y,'y')
    if not status:
        return y
    df=alphaFunc.correlation(x,y)
    return np.nan_to_num(df.values)

def _covariance(x,y):
    status,x=trans_xy(x,'x')
    if not status:
        return x
    status,y=trans_xy(y,'y')
    if not status:
        return y
    df=alphaFunc.covariance(x,y)
    return np.nan_to_num(df.values)

cov=gp.functions.make_function(function = _covariance,name = 'covariance',arity = 2)
corr=gp.functions.make_function(function = _correlation,name = 'correlation',arity = 2)
log=gp.functions.make_function(function = _log,name = 'log',arity = 1)
ts_sum=gp.functions.make_function(function = _ts_sum,name = 'ts_sum',arity = 1)
delta=gp.functions.make_function(function = _delta,name = 'delta',arity = 1)

my_function=[cov,corr,log,ts_sum,delta]
init_function = ['add', 'sub']
gp1 = SymbolicTransformer(  
                            generations=3, #整数，可选(默认值=20)要进化的代数
                            population_size=1000,# 整数，可选(默认值=1000)，每一代群体中的公式数量
                            hall_of_fame=30, 
                            n_components=30,#最终生成的因子数量
                            function_set=init_function+my_function,
                            parsimony_coefficient=0.0005,
                            max_samples=0.9, 
                            verbose=1,
                            const_range = (10.0,20.0),
                            feature_names=list('$'+n for n in train.columns),
                            random_state=int(time.time()),  # 随机数种子
                            n_jobs=-1
                    )

gp1.fit(train,label)
print(gp1)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    12.31         0.013487        4         0.108429        0.0526303      1.18m
   1     4.89         0.043503       10         0.133198         0.115304     21.38s
   2     4.28        0.0817211       10         0.156221         0.130051      0.00s
[sub(ts_sum(sub(ts_sum($buySmVol_0), $close)), ts_sum(ts_sum(ts_sum($buySmVol_0)))),
 ts_sum(ts_sum(ts_sum($buySmVol_0))),
 ts_sum(sub(ts_sum(sub(ts_sum($buySmVol_0), $close)), ts_sum(add($buySmVol_0, $low)))),
 ts_sum(add(ts_sum(ts_sum($buySmVol_0)), $buySmVol_0)),
 ts_sum(sub(sub(ts_sum(sub(ts_sum($buySmVol_0), $close)), ts_sum(add($buySmVol_0, $low))), $low)),
 ts_sum(ts_sum(ts_sum($buySmVol_0))),
 sub(ts_sum(sub(ts_sum($buySmVol_0), $close)), ts_sum(ts_sum(ts_sum($buySmVol_0)))),
 sub(correl